In [47]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.question_answering import load_qa_chain

import os


In [48]:
# Load dos modelos (Embeddings e LLM)
embeddings_model = OpenAIEmbeddings()
llm = ChatOpenAI(model_name="gpt-4o", max_tokens=200)

# Carregar o PDF
pdf_link = "example2.pdf"

loader = PyPDFLoader(pdf_link, extract_images=False)
pages = loader.load_and_split()

# Separar em Chunks (Pedaços de documento)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=4000,  # Define o tamanho máximo de cada chunk em 4000 caracteres
    chunk_overlap=20,  # Define uma sobreposição de 20 caracteres entre os chunks consecutivos
    length_function=len,  # Usa a função len para calcular o tamanho dos chunks com base no número de caracteres
    add_start_index=True  # Adiciona o índice de início em cada chunk para rastrear a posição original no documento
)

chunks = text_splitter.split_documents(pages)

In [49]:
#Salvar no Vector DB - Chroma

db = Chroma.from_documents(chunks, embedding=embeddings_model, persist_directory="new_index")
db.persist()

In [50]:
#Carregar DB
vectordb = Chroma(persist_directory="new_index", embedding_function=embeddings_model)

#Load Retriever
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

#Construção da cadeia de prompt para chamada do LLM
chain = load_qa_chain(llm, chain_type="stuff")

In [51]:
def ask(question):
    context = retriever.get_relevant_documents(question)
    answer = (chain({"input_documents": context, "question": question}, return_only_outputs=True))['output_text']
    return answer, context

In [56]:
user_question = input("User: ")
answer, context = ask(user_question)
print("Answer: ", answer)

Answer:  Eu não sei exatamente qual solução de RAG você está se referindo. No entanto, RAG geralmente significa "Retrieval-Augmented Generation", uma técnica usada em processamento de linguagem natural que combina recuperação de informações com geração de texto. Se precisar de mais informações específicas ou um exemplo de implementação, recomendo consultar materiais especializados ou recursos relacionados.


In [57]:
print("Context: ", context[0])

Context:  page_content='Rocketseat
Todos os direitos reservados#BoostingPeople
rocketseat.com.br
Arquitetura simples 
de RAG' metadata={'page': 34, 'source': 'example.pdf', 'start_index': 0}
